# Databearbetning Projekt 1
### Markus Kankkonen


## Uppgift 1.
- Importera våran data ta ent titt på datan

Vi börjar med att importera Pandas

In [1]:
import pandas as pd


Vi importerar våra filer

In [2]:
movieRating = pd.read_csv("u.data",sep="\t",names=["UserID","ItemID","rating","timestamp"])
movieData = pd.read_csv("u.item",sep="|",encoding="latin-1",names=["ItemID","Name","Date","VideoReleaseDate","IMDB","Unknown","Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"])
movieUsers = pd.read_csv("u.user",sep="|",encoding="latin-1",names=["UserID","Age","Gender","Occupation","ZIP-Code"])


Vi tar en titt på vår data med describe, dtypes och head

In [3]:
movieRating.describe()


,UserID,ItemID,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [4]:
movieData.dtypes

ItemID                int64
Name                 object
Date                 object
VideoReleaseDate    float64
IMDB                 object
Unknown               int64
Action                int64
Adventure             int64
Animation             int64
Children's            int64
Comedy                int64
Crime                 int64
Documentary           int64
Drama                 int64
Fantasy               int64
Film-Noir             int64
Horror                int64
Musical               int64
Mystery               int64
Romance               int64
Sci-Fi                int64
Thriller              int64
War                   int64
Western               int64
dtype: object

In [5]:
movieUsers.head()

,UserID,Age,Gender,Occupation,ZIP-Code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


## Uppgift 2.
För mycket data!!! 
Vi filtrerar bort allt onödigt

-Endast kön, ålder och yrke av användarna

In [6]:
movieUsers[["Age","Gender","Occupation"]]

,Age,Gender,Occupation
0,24,M,technician
1,53,F,other
2,23,M,writer
3,24,M,technician
4,33,F,other
...,...,...,...
938,26,F,student
939,32,M,administrator
940,20,M,student
941,48,F,librarian


- Gruppera ratings enligt film

Här stötte jag på lite problem med att förstå mig på hur merge fungerar. Koden fungerar bra men de känns bara lite flummigt :)

In [7]:
# Vi slår ihop movieData med movieRating via ItemID.
moveBy = movieRating.merge(movieData, on="ItemID",how="inner")
# Från den nya listan tar vi ut Name och rating
ratingPerMovie = moveBy[["Name","rating"]]
ratingPerMovie

,Name,rating
0,Kolya (1996),3
1,Kolya (1996),3
2,Kolya (1996),5
3,Kolya (1996),3
4,Kolya (1996),5
...,...,...
99995,Mamma Roma (1962),4
99996,"Eighth Day, The (1996)",3
99997,Girls Town (1996),3
99998,"Silence of the Palace, The (Saimt el Qusur) (1...",3


### Uppgift 3.
-Visa endast användare med ett visst yrke

In [8]:
# Alla movieUsers som innehåller strängen student
movieUsers[movieUsers["Occupation"].str.contains("student")]

,UserID,Age,Gender,Occupation,ZIP-Code
8,9,29,M,student,01002
29,30,7,M,student,55436
31,32,28,F,student,78741
32,33,23,M,student,27510
35,36,19,F,student,93117
...,...,...,...,...,...
927,928,21,M,student,55408
932,933,28,M,student,48105
938,939,26,F,student,33319
940,941,20,M,student,97229


In [36]:
# Alla Gender == M och Age större än 40
malesOver40 = movieUsers[movieUsers["Age"].gt(40) & movieUsers["Gender"].str.contains("M")]
malesOver40

,UserID,Age,Gender,Occupation,ZIP-Code
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
9,10,53,M,lawyer,90703
12,13,47,M,educator,29206
13,14,45,M,scientist,55106
...,...,...,...,...,...
930,931,60,M,educator,33556
931,932,58,M,educator,06437
933,934,61,M,engineer,22902
934,935,42,M,doctor,66221


### Uppgift 4
- Medelåldern av användare som är författare

In [37]:
# Mean på åldern av alla Occupation == writer
meanOfWriters = movieUsers[movieUsers["Occupation"].str.contains("writer")]["Age"].mean()
print("Mean age of writers",meanOfWriters)

Mean age of writers 36.31111111111111


- Medelrating per film

Jag skrev in map funktionerna och förväntade mig att det int skulle fungera men till min förvåning så fungera allt från första början. Lite osäker än på hur map funktionen verkligen fungerar med DataFrames. Det här borde säkert gå att lösa på ett bättre sätt en att mappa till alla rows och sen drop_duplicates() men hej de fungerar åtminstone

In [12]:
# Kopiera dataframen så vi inte får SettingWithCopyWarning
ratingCopy = ratingPerMovie.copy()
# Sätt till nya kolumner till Dataframen med att mappa igenom dataframen
ratingCopy["Mean"] = ratingPerMovie.Name.map(ratingPerMovie.groupby(["Name"]).rating.mean())
ratingCopy["Reviews"] = ratingPerMovie.Name.map(ratingPerMovie.groupby(["Name"]).Name.count())
# Nu har varje rating på samma film ett mean värde. Ta bort duplikat 
ratingCopy = ratingCopy.drop_duplicates("Name")
# Returnera Name Mean och Review
ratingCopy[["Name","Mean","Reviews"]].head(10).style.hide_index()

Name,Mean,Reviews
Kolya (1996),3.991453,117
L.A. Confidential (1997),4.161616,297
Heavyweights (1994),2.153846,13
Legends of the Fall (1994),3.456790,81
Jackie Brown (1997),3.642857,126
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),4.252577,194
"Hunt for Red October, The (1990)",3.863436,227
"Jungle Book, The (1994)",3.564706,85
Grease (1978),3.347059,170
"Remains of the Day, The (1993)",3.940000,150


In [13]:
# Ta alla som har Count större än 40 och sortera med Mean kolumnen
top10 = ratingCopy[ratingCopy["Reviews"].ge(40)].sort_values(by="Mean",ascending=False)
top10[["Name","Mean"]].head(10).style.hide_index()

Name,Mean
"Close Shave, A (1995)",4.491071
Schindler's List (1993),4.466443
"Wrong Trousers, The (1993)",4.466102
Casablanca (1942),4.456790
Wallace & Gromit: The Best of Aardman Animation (1996),4.447761
"Shawshank Redemption, The (1994)",4.445230
Rear Window (1954),4.387560
"Usual Suspects, The (1995)",4.385768
Star Wars (1977),4.358491
12 Angry Men (1957),4.344000


## Uppgift 5.
- Följande Dataframen visar vilka yrken de olika könen har 
- Differance variabeln är mängden män - mängden kvinnor som har samma job. Ett nummer så nära 0 som möjligt betyder att det är lika mycket. Ett positivt värde betyder flera män och ett negativt flera kvinnor.
- Total variabeln är den total mängden av personer som arbetar med ett yrke.
- Percentage visar procentuella skillnade 100% ändast män 0% ändast kvinnor och 50% e 50/50

Det här tog en stund. Känns som att det måste vara möjligt att göra detta på ett lättare sätt med mycket mindre kod. Men jag hade svårt att få allt och fungera så jag beslöt mig för att göra en sak åt gången.

In [34]:
# Separera dataframen till män och kvinnor
females = movieUsers[movieUsers["Gender"].str.contains("F")]
males = movieUsers[movieUsers["Gender"].str.contains("M")]

# Det finns olika mängder arbeten på listorna 
mJobs = males["Occupation"].value_counts()
fJobs = females["Occupation"].value_counts()
# Ta bort de yrken som ändast finns på männens sida
fbool = fJobs.index.to_series()
mbool = mJobs.index.to_series()
mJobs = mJobs[mbool.isin(fbool)]

#  Räkna ut differencen och totala mängden samt procent
diff = mJobs - fJobs
tot = mJobs + fJobs
procent = (mJobs / tot ) 
procent = pd.Series(["{0:.2f}%".format(val * 100) for val in procent], index = procent.index)

procent = procent.rename("Percentage")
diff = diff.rename("Difference")
tot = tot.rename("Total")
mJobs = mJobs.rename("Male")
fJobs = fJobs.rename("Female")

# Gör en DataFrame av våra serier
jobs = [mJobs,fJobs,tot,diff,procent]
jobs = pd.DataFrame(jobs)
# Transpose
jobs.T


,Male,Female,Total,Difference,Percentage
student,136,60,196,76,69.39%
other,69,36,105,33,65.71%
educator,69,26,95,43,72.63%
engineer,65,2,67,63,97.01%
programmer,60,6,66,54,90.91%
administrator,43,36,79,7,54.43%
executive,29,3,32,26,90.62%
scientist,28,3,31,25,90.32%
technician,26,1,27,25,96.30%
writer,26,19,45,7,57.78%


 - Kombinera u.data med u.item för att skapa en tabell där man kan se movie title och rating
 

In [18]:
# Det här gjorde vi ju redan tidigare
moveBy

,UserID,ItemID,rating,timestamp,Name,Date,VideoReleaseDate,IMDB,Unknown,Action,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
1,63,242,3,875747190,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
2,226,242,5,883888671,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
3,154,242,3,879138235,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
4,306,242,5,876503793,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,840,1674,4,891211682,Mamma Roma (1962),01-Jan-1962,NaN,http://us.imdb.com/M/title-exact?Mamma%20Roma%...,0,0,...,0,0,0,0,0,0,0,0,0,0
99996,655,1640,3,888474646,"Eighth Day, The (1996)",01-Nov-1996,NaN,"http://us.imdb.com/Title?Huiti%E8me+jour,+Le+(...",0,0,...,0,0,0,0,0,0,0,0,0,0
99997,655,1637,3,888984255,Girls Town (1996),23-Aug-1996,NaN,http://us.imdb.com/M/title-exact?Girls%20Town%...,0,0,...,0,0,0,0,0,0,0,0,0,0
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1...",02-Feb-1996,NaN,http://us.imdb.com/M/title-exact?Saimt%20el%20...,0,0,...,0,0,0,0,0,0,0,0,0,0


- Kombinera datan med users för att se vem som har sett vilken film

In [16]:
# Merge in movieUsers
userRatingMovie = moveBy.merge(movieUsers, on="UserID",how="inner")
userRatingMovie[["UserID","Occupation","Name","rating"]]

,UserID,Occupation,Name,rating
0,196,writer,Kolya (1996),3
1,196,writer,Men in Black (1997),2
2,196,writer,"Truth About Cats & Dogs, The (1996)",4
3,196,writer,"Birdcage, The (1996)",4
4,196,writer,"Adventures of Priscilla, Queen of the Desert, ...",4
...,...,...,...,...
99995,873,administrator,Titanic (1997),5
99996,873,administrator,G.I. Jane (1997),4
99997,873,administrator,Desperate Measures (1998),3
99998,873,administrator,Spawn (1997),2


- Vem är de snällaste och tuffaste reviewers!!

In [47]:
# Mappa in mean värden
userRatingMovie["Mean"] = userRatingMovie.UserID.map(userRatingMovie.groupby(["UserID"]).rating.mean())
# Bort me duplicates
uniqueUsers = userRatingMovie.drop_duplicates("UserID")
# Sortera
uu = uniqueUsers[["UserID","Occupation","Mean"]].sort_values(by="Mean",ascending=False)
uu


,UserID,Occupation,Mean
96954,849,student,4.869565
37966,688,administrator,4.833333
24785,507,writer,4.724138
8995,628,none,4.703704
98285,928,student,4.687500
...,...,...,...
80493,774,student,2.058036
26992,685,librarian,2.050000
30922,445,writer,1.985185
20479,405,healthcare,1.834464


### Conclusion
efter att ha sett på head och tail värden en del så kan ja säga att det finns mycket studenter som är både snälla och tuffa.

## 6. Reflektion och feedback
Pandas e helt kul. Det är så mycket abstraktion att man emellanåt känner sig lite overwhelmed. Men det känns som ett väldigt kraftfullt verktyg. 